In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from sensor_projection_utils import *
from datetime import  datetime
import os
import matplotlib.pyplot as plt

In [2]:
def video_frames_value_matrix(video_number,frames_range,data_type='X'):
    
    
    
    
    column_names = [f"U{i}.{data_type}" for i in range(1, 49)]

    # Create an empty DataFrame with specified columns
    matrix = pd.DataFrame(columns=column_names)

    # Add the index column "Frame.No" with integer values
    matrix.index.name = 'Frame.No'

    
    f1 = frames_range[0]
    f2 = frames_range[0] 
    # Determine the number of rows based on frames_range
    num_rows = frames_range[1] - frames_range[0] + 1
    
    
     # Iterate through user folders
    for user_number in range(1, 49):
        folder_path = f"./{user_number}/"  # Update this path
        
        # Check if folder exists
        if os.path.exists(folder_path):
            csv_path = os.path.join(folder_path, f"video_{video_number}-projection.csv")
            # Check if CSV file exists
            if os.path.exists(csv_path):
                # Read CSV file
                df = pd.read_csv(csv_path)
                df.drop_duplicates(subset=['Frame.No'], inplace=True)
                trunc_df = df.loc[(df['Frame.No'] >= frames_range[0]) & (df['Frame.No'] <= frames_range[1]), ['Frame.No', data_type]]
                matrix[f'U{user_number}.X'] = trunc_df.set_index('Frame.No')[data_type]
    
    ###preprocessing 
    matrix = matrix.fillna(method='ffill')
    matrix['Mean'] = matrix.mean(axis=1)
    matrix['Std_Dev'] = matrix.std(axis=1)
    return matrix

In [4]:
mat1_onemin_x = video_frames_value_matrix(1,[1,1800],data_type='X')


In [5]:
mat1_onemin_x

,U1.X,U2.X,U3.X,U4.X,U5.X,U6.X,U7.X,U8.X,U9.X,U10.X,...,U41.X,U42.X,U43.X,U44.X,U45.X,U46.X,U47.X,U48.X,Mean,Std_Dev
Frame.No,,,,,,,,,,,,,,,,,,,,,
1,0.644603,1.003681,0.948804,0.956895,0.389074,0.726045,0.269375,0.460864,0.517021,0.369413,...,0.777438,0.741573,0.308448,0.773259,0.531438,1.226232,1.023273,1.239742,0.734093,0.283696
2,0.644587,1.003643,0.948089,0.954611,0.389074,0.725722,0.270309,0.460864,0.516999,0.369413,...,0.797238,0.741573,0.308448,0.772289,0.531902,1.244445,1.023067,1.239744,0.734778,0.284248
3,0.644587,1.003863,0.947893,0.953434,0.389074,0.725381,0.271533,0.460864,0.516991,0.369434,...,0.778098,0.741573,0.308177,0.772937,0.531826,1.226232,1.022840,1.239744,0.734037,0.283533
4,0.644581,1.003643,0.947884,0.926970,0.409882,0.725398,0.268896,0.460864,0.516542,0.369413,...,0.780463,0.741824,0.311768,0.772944,0.531846,1.226193,1.022853,1.239744,0.734232,0.282479
5,0.644576,1.003643,0.948326,0.952044,0.389361,0.725385,0.272482,0.460864,0.516047,0.369729,...,0.780848,0.741231,0.308539,0.772616,0.531846,1.226519,1.022866,1.239744,0.734094,0.283504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,0.300606,0.461218,0.528331,0.533839,0.297308,0.329153,0.527314,0.350732,0.279119,0.291221,...,0.371148,0.375104,0.338738,0.306442,0.443555,0.278949,0.288925,0.316423,0.394218,0.157131
1797,0.300606,0.459591,0.526768,0.533826,0.297308,0.329153,0.527314,0.348349,0.278446,0.291782,...,0.371148,0.375458,0.339583,0.306104,0.441929,0.280193,0.288706,0.316423,0.394741,0.158332
1798,0.300541,0.458718,0.525835,0.533578,0.297600,0.328860,0.524805,0.347288,0.278498,0.291782,...,0.371148,0.375779,0.341482,0.306104,0.440754,0.281480,0.288571,0.316728,0.394929,0.158920


In [6]:
def matrix_variations(df):
    # Initialize an empty DataFrame to store differences
    df = df.drop(columns=['Mean','Std_Dev'])
    diff_df = pd.DataFrame(index=[f'{i}-{i+1}' for i in range(1, len(df))], columns=df.columns)

    # Iterate over rows
    for i in range(1, len(df)):
        # Calculate differences between consecutive rows
        row_diff = df.iloc[i] - df.iloc[i - 1]
        
        # Store differences in the new DataFrame
        diff_df.iloc[i - 1] = row_diff.values  # Include all columns
    
    diff_df['Mean'] = diff_df.mean(axis=1)
    diff_df['Std_Dev'] = diff_df.std(axis=1)
        
    return diff_df


In [7]:
matrix_variations(mat1_onemin_x)

,U1.X,U2.X,U3.X,U4.X,U5.X,U6.X,U7.X,U8.X,U9.X,U10.X,...,U41.X,U42.X,U43.X,U44.X,U45.X,U46.X,U47.X,U48.X,Mean,Std_Dev
1-2,-0.000016,-0.000038,-0.000715,-0.002284,0.0,-0.000324,0.000934,0.0,-0.000022,0.0,...,0.0198,0.0,0.0,-0.00097,0.000464,0.018213,-0.000206,0.000002,0.000684,0.004249
2-3,0.0,0.00022,-0.000196,-0.001177,0.0,-0.000341,0.001224,0.0,-0.000008,0.00002,...,-0.019141,0.0,-0.000271,0.000649,-0.000076,-0.018213,-0.000228,0.0,-0.000741,0.004163
3-4,-0.000006,-0.00022,-0.00001,-0.026463,0.020808,0.000018,-0.002637,0.0,-0.000449,-0.00002,...,0.002366,0.000251,0.003591,0.000006,0.000019,-0.000039,0.000013,0.0,0.000196,0.005251
4-5,-0.000005,0.0,0.000442,0.025073,-0.020521,-0.000013,0.003586,0.0,-0.000494,0.000316,...,0.000385,-0.000593,-0.003229,-0.000328,0.0,0.000326,0.000013,0.0,-0.000138,0.005091
5-6,0.00029,0.0,-0.058002,-0.027397,-0.000063,0.0,0.00092,-0.000251,-0.016985,0.029841,...,0.000723,0.0,-0.000362,0.0,0.000057,0.000287,-0.043798,-0.000319,-0.001787,0.013510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719-1720,-0.000039,-0.000827,-0.000895,0.0,0.000278,0.0,-0.0015,-0.00117,0.000022,0.000272,...,-0.000018,0.000073,0.001355,-0.000127,-0.002576,0.001206,-0.000042,0.0,0.000340,0.001547
1720-1721,0.0,-0.001627,-0.001563,-0.000013,0.0,0.0,0.0,-0.002382,-0.000674,0.000561,...,0.0,0.000354,0.000845,-0.000338,-0.001626,0.001244,-0.00022,0.0,0.000523,0.002024
1721-1722,-0.000065,-0.000873,-0.000933,-0.000248,0.000292,-0.000292,-0.002509,-0.001061,0.000053,0.0,...,0.0,0.000321,0.001899,0.0,-0.001174,0.001287,-0.000134,0.000305,0.000188,0.001466
1722-1723,-0.000039,-0.001347,-0.001642,-0.0002,0.000292,0.0,-0.003199,0.0,0.000104,0.000454,...,0.000059,0.000243,0.001895,0.000215,0.0,0.002734,0.0,0.0,0.000393,0.002418


In [ ]:
def draw_matrix(df, user_range, color_map=None):
    """
    Draw Ui.X columns from DataFrame df for the specified user range.

    Parameters:
    - df: DataFrame containing the columns to be drawn.
    - user_range: Range of user numbers to draw (e.g., (1, 10)).
    - color_map: Dictionary mapping user numbers to colors (optional).
    """
    # Extract the relevant columns for the specified user range
    user_columns = [f'U{user}.X' for user in range(user_range[0], user_range[1] + 1)]
    df_subset = df[user_columns]

    # Set up colors for the plots
    if color_map is None:
        color_map = {f'U{user}.X': plt.cm.tab10(i) for i, user in enumerate(range(user_range[0], user_range[1] + 1))}

    # Plot each column with specified color
    for column in df_subset.columns:
        plt.plot(df.index, df_subset[column], label=column, color=color_map[column])

    # Add labels and legend
    plt.xlabel('Frame.No')
    plt.ylabel('Value X')
    plt.legend(title='User', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Show the plot
    plt.show()